In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
import subprocess
import sys

# Path del binario del genético
binario = '../bin/genetic_trainer'

# Cosas que no tocaremos
n = 7
m = 6
c = 4
p = str(n*m*2)
generaciones = 50
cross = 'c:completo'
mut = 'm:multi'

# Para crear el comando que ejecuta el genético
def comando_genetico(fitness, seleccion,pob,p_mut):
    res = [binario] + [str(n)]+[str(m)]+[str(c)]+[str(p)]
    res = res+ [str(generaciones)] +[str(pob)]+[str(p_mut)]
    res = res+[fitness]+[seleccion]+[cross]+[mut]
    return res

# Para ejecutar subprocess en shell con stdout live
def shell(command):
    p = subprocess.Popen(command, stdout=subprocess.PIPE, bufsize=1)
    for line in iter(p.stdout.readline, b''):
        sys.stdout.write(line)
    p.stdout.close()
    p.wait()

        
# Para guardar los logs

# Leo los fitness, los formateo en lista
def save_logs(path):
    with open('log/best_fitness_log.dat') as file:
        lista = file.read().splitlines()
    fitness = list(map(float, lista))

    # Leo los progenitores, los formateo en lista 
    with open('log/progenitores.dat') as file:
        lista = file.read().splitlines()
    for i in range(len(lista)):
        lista[i] = lista[i].split(',')
        lista[i] = list(map(float, lista[i]))
    progenitores = lista

    # Leo el ganador
    with open('log/ganador.dat') as file:
        lista = file.read().splitlines()
    ganador_fitness = float(lista[0])
    ganador_genoma = list(map(float, lista[1].split(',')))

    # Guardo todo como una super tupla
    np.save(path,(fitness,progenitores,ganador_fitness,ganador_genoma))

## Cambiando proba mutacion

In [ ]:
fit = 'f:vs_random'
sel = 's:rank'
pob = 10
p_mut_s_pablo = [0.5,0.25,0.1,0.05]
p_mut_s_nico = [0.02,0.005]
p_mut_filename_pablo = ["save/genetico/exp_p_mut_" + str(pm) + ".npy" for pm in p_mut_s_pablo]
p_mut_filename_nico = ["save/genetico/exp_p_mut_" + str(pm) + ".npy" for pm in p_mut_s_nico]

## PABLO

In [ ]:
for i in range(len(p_mut_s_pablo)):
    command = comando_genetico(fit,sel,pob,p_mut_s_pablo[i])    
    print("LLAMANDO A: " + reduce(lambda x,y: x +" "+y, command))
    # Ejecuto el entrenamiento. En ./log/ me deja los .dat como logs.
    shell(command)
    # Guardo los logs
    save_logs(p_mut_filename_pablo[i])    

## NICO

In [ ]:
for i in range(len(p_mut_s_nico)):
    command = comando_genetico(fit,sel,pob,p_mut_s_nico[i])    
    # Ejecuto el entrenamiento. En ./log/ me deja los .dat como logs.
    shell(command)
    # Guardo los logs
    save_logs(p_mut_filename_nico[i])  

In [45]:
plt.rcParams.update({'font.size': 23})
# Ploteo todas las curvas
plt.figure(figsize=(12,10))
for i in range(len(p_mut_s_nico)+len(p_mut_s_pablo)):
    (fitness,progenitores,ganador_fitness,ganador_genoma) = np.load((p_mut_filename_nico+p_mut_filename_pablo)[i])
    plt.plot(fitness,label=str((p_mut_s_nico+p_mut_s_pablo)[i]))

plt.legend(loc='best')
plt.xlabel('Generacion')
plt.xlim(0,50)
plt.ylabel('Fitness')
# plt.show()
plt.tight_layout()
plt.savefig("plots/genetico/exp_p_mutacion.pdf")
# plt.show()


In [46]:
compare_label = "save/genetico/exp_p_mut_0.1.npy"
(fitness_sub,progenitores,ganador_fitness,ganador_genoma) = np.load(compare_label)
names = (p_mut_filename_nico+p_mut_filename_pablo)
plt.figure(figsize=(12,10))
for i in range(len(p_mut_s_nico)+len(p_mut_s_pablo)):
    if compare_label != names[i]:
        (fitness,progenitores,ganador_fitness,ganador_genoma) = np.load((p_mut_filename_nico+p_mut_filename_pablo)[i])
        plt.plot(np.array(fitness)-np.array(fitness_sub),label=str((p_mut_s_nico+p_mut_s_pablo)[i]))
plt.legend(loc='best')
plt.xlabel('Generacion')
plt.xlim(0,50)
plt.ylabel('Fitness')
# plt.show()
plt.tight_layout()
plt.axhline(0, color="red")
# plt.show()
plt.savefig("plots/genetico/p_mutacion_vs_.1.pdf")
